In [1]:
# import req'd modules
import json
import os
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import pmagpy.builder as builder

## Playing with json & unicode

In [2]:

# json is the same format that the MagIC data model comes in

# turn json into Python:
json_string = '{"first_name": "Guido", "last_name":"Rossum"}'
parsed = json.loads(json_string)

# turn Python into json
d = {'hello': 'hi', 'so long': 'goodbye'}
dumped = json.dumps(d)

# store json in a file
outfile = open('stored.json', 'w')
json.dump(dumped, outfile)
outfile.close()

# read json file into Python
jstring = json.load(open('stored.json', 'r'))
json.loads(jstring)

{u'hello': u'hi', u'so long': u'goodbye'}

In [3]:
# parsing unicode
unicode('\r\n\n\xc2\xa0\xc2\xa0', errors='ignore')

u'\r\n\n'

## Reading the data model in to pandas

In [4]:
# the code in this block has been incorporated into data_model3.py

def get_data_model():
    model_file = os.path.join('3_0', 'MagIC Data Model v3.0 - unpublished.json')
    f = open(model_file, 'r')
    string = '\n'.join(f.readlines())
    raw = json.loads(unicode(string, errors='ignore'))
    full = DataFrame(raw)
    return full
    
full = get_data_model()
DataFrame(full['tables']['locations'])
location = DataFrame(full['tables']['locations']['columns'])
location = location.transpose()
#full['tables']['locations'].pop('columns')
#full['tables']['locations']
# don't really need anything that isn't in ['tables'][table]['columns']
location[:3]

full_df = get_data_model()

data_model = {}
levels = ['specimens', 'samples', 'sites', 'locations', 'criteria']
for level in levels:
    df = DataFrame(full_df['tables'][level]['columns'])
    data_model[level] = df.transpose()

data_model['sites']

,description,examples,group,label,notes,position,previous_columns,type,unit,urls,validations
age,Site inferred age,NaN,Age,Inferred Age,NaN,29,"[{u'column': u'site_inferred_age', u'table': u...",Number,Custom,NaN,"[requiredUnless(""age_low"",""age_high""), require..."
age_high,"Site inferred age, High range",NaN,Age,Inferred Age High,NaN,32,"[{u'column': u'site_inferred_age_high', u'tabl...",Number,Custom,NaN,"[min(""age_low""), requiredUnless(""age"")]"
age_low,"Site inferred age, Low range",NaN,Age,Inferred Age Low,NaN,31,"[{u'column': u'site_inferred_age_low', u'table...",Number,Custom,NaN,"[max(""age_high""), requiredUnless(""age"")]"
age_sigma,"Site inferred age, Uncertainty",NaN,Age,Inferred Age Sigma,Standard error or standard deviation at one sigma,30,"[{u'column': u'site_inferred_age_sigma', u'tab...",Number,Custom,NaN,[min(0)]
age_unit,"Site inferred age, Unit",NaN,Age,Inferred Age Unit,NaN,33,"[{u'column': u'site_inferred_age_unit', u'tabl...",String,NaN,NaN,"[cv(""age_unit""), required()]"
analyst_names,Colon-delimited list of EarthRef handles and/o...,[@user1:@user2:Not A. Member <no.earthref.hand...,Metadata,Analyst Names,NaN,112,"[{u'column': u'er_analyst_mail_names', u'table...",List,NaN,NaN,"[type(""users"")]"
aniso_f,"Foliation, T2/T3",NaN,Anisotropy,Anisotropy F,NaN,97,"[{u'column': u'anisotropy_f', u'table': u'rmag...",Number,Dimensionless,NaN,NaN
aniso_ff,"Log foliation, ln(F)",NaN,Anisotropy,Anisotropy F',NaN,99,"[{u'column': u'anisotropy_ff', u'table': u'rma...",Number,Dimensionless,[http://dx.doi.org/10.1130/0016-7606(1977)88<1...,NaN
aniso_fl,F/L,NaN,Anisotropy,Anisotropy FL Ratio,NaN,101,"[{u'column': u'anisotropy_fl', u'table': u'rma...",Number,Dimensionless,NaN,NaN
aniso_ftest,F statistical test for anisotropy,NaN,Anisotropy,Anisotropy F Test,NaN,102,"[{u'column': u'anisotropy_ftest', u'table': u'...",Number,%,NaN,NaN


## Extracting info from the data model

In [5]:
# how to get various different data from the data model

# get all headers of a particular group
cond = location['group'] == 'Age'
age_columns = location[cond]
age_columns

,description,examples,group,label,notes,position,previous_columns,type,unit,urls,validations
age_high,"Location inferred age, High range",NaN,Age,Inferred Age High,NaN,36,"[{u'column': u'average_age_high', u'table': u'...",Number,Custom,NaN,"[min(""age_low""), requiredUnlessTable(""sites"")]"
age_low,"Location inferred age, Low range",NaN,Age,Inferred Age Low,NaN,35,"[{u'column': u'average_age_low', u'table': u'p...",Number,Custom,NaN,"[max(""age_high""), requiredUnlessTable(""sites"")]"
age_unit,"Location inferred age, Age unit",NaN,Age,Inferred Age Unit,NaN,37,"[{u'column': u'average_age_unit', u'table': u'...",String,NaN,NaN,"[cv(""age_unit""), requiredUnlessTable(""sites"")]"


In [6]:
# get a particular column
location.ix['age_high']

description                         Location inferred age, High range
examples                                                          NaN
group                                                             Age
label                                               Inferred Age High
notes                                                             NaN
position                                                           36
previous_columns    [{u'column': u'average_age_high', u'table': u'...
type                                                           Number
unit                                                           Custom
urls                                                              NaN
validations            [min("age_low"), requiredUnlessTable("sites")]
Name: age_high, dtype: object

In [7]:
# get validations for a particular column
validations = location.ix['age_high']['validations']
validations

[u'min("age_low")', u'requiredUnlessTable("sites")']

In [8]:
# get all groups for locations
location['group'].unique()

array([u'Age', u'Metadata', u'Result', u'Direction', u'Position',
       u'Expedition', u'Names', u'Geology', u'Location', u'PADM',
       u'Paleoposition', u'PDM', u'Pole'], dtype=object)

In [9]:
# get all rows in a group
group = 'Direction'
location[location['group'] == group]

,description,examples,group,label,notes,position,previous_columns,type,unit,urls,validations
conglomerate_test,Classification and result of the (intra-format...,NaN,Direction,Conglomerate Test,NaN,50,"[{u'column': u'conglomerate_test', u'table': u...",String,Flag,NaN,"[cv(""conglomerate_test"")]"
contact_test,Classification and result of the (inverse) con...,NaN,Direction,Baked Contact Test,NaN,51,"[{u'column': u'contact_test', u'table': u'pmag...",String,Flag,NaN,"[cv(""contact_test"")]"
dic_inc,Location direction in coordinates specified by...,NaN,Direction,Direction Inclination,NaN,40,"[{u'column': u'average_inc', u'table': u'pmag_...",Number,Degrees,NaN,"[min(-90), max(90), requiredIfGroup(""Direction"")]"
dir_alpha95,Location direction in coordinates specified by...,NaN,Direction,Direction Alpha 95%,Confidence Level = 95%,41,"[{u'column': u'average_alpha95', u'table': u'p...",Number,Degrees,NaN,[min(0)]
dir_dec,Location direction in coordinates specified by...,NaN,Direction,Direction Declination,NaN,39,"[{u'column': u'average_dec', u'table': u'pmag_...",Number,Degrees,NaN,"[min(0), max(360), requiredIfGroup(""Direction"")]"
dir_k,Location direction in coordinates specified by...,NaN,Direction,Direction K,NaN,43,"[{u'column': u'average_k', u'table': u'pmag_re...",Number,Dimensionless,NaN,[min(0)]
dir_k_ratio,Comparison of Fisher dispersion K after and be...,NaN,Direction,Direction Tilt K Ratio,NaN,44,"[{u'column': u'tilt_k_ratio', u'table': u'pmag...",Number,Dimensionless,NaN,[min(0)]
dir_n_samples,Number of samples included in directional calc...,NaN,Direction,Direction N Samples,NaN,46,"[{u'column': u'average_nn', u'table': u'pmag_r...",Integer,NaN,NaN,[min(0)]
dir_n_sites,Number of sites included in directional calcul...,NaN,Direction,Direction N Sites,NaN,45,"[{u'column': u'average_n', u'table': u'pmag_re...",Integer,NaN,NaN,[min(0)]
dir_polarity,"Location direction polarity is normal (n), rev...",NaN,Direction,Direction Polarity,NaN,47,NaN,String,Flag,NaN,"[cv(""polarity"")]"


In [10]:
# get all column labels for locations
print list(location.index)
print 'required()' in location.ix['location_name']['validations']
print 'required()' in location.ix['continent_ocean']['validations']

[u'age_high', u'age_low', u'age_unit', u'analyst_names', u'citations', u'conglomerate_test', u'contact_test', u'continent_ocean', u'country', u'criteria_names', u'description', u'dic_inc', u'dir_alpha95', u'dir_dec', u'dir_k', u'dir_k_ratio', u'dir_n_samples', u'dir_n_sites', u'dir_polarity', u'dir_r', u'dir_tilt_correction', u'elevation_high', u'elevation_low', u'expedition_description', u'expedition_leg', u'expedition_name', u'expedition_ship', u'expedition_url', u'experiment_names', u'external_database_ids', u'fold_test', u'fold_test_significance', u'geologic_classes', u'geological_province_sections', u'lat_n', u'lat_s', u'lithologies', u'location_name', u'location_name_alternatives', u'location_type', u'lon_e', u'lon_w', u'method_codes', u'ocean_sea', u'padm', u'padm_n_sites', u'padm_sigma', u'paleolat', u'paleolat_sigma', u'paleolon', u'paleolon_sigma', u'pdm', u'pdm_n_sites', u'pdm_sigma', u'pi_names', u'plate_blocks', u'pole_alpha95', u'pole_antipodal_angle', u'pole_bc_q', u'pol

In [11]:
# get list of unique groups for location
print location['group'].unique()

#sort column names by group
location.sort_values('group').head()

[u'Age' u'Metadata' u'Result' u'Direction' u'Position' u'Expedition'
 u'Names' u'Geology' u'Location' u'PADM' u'Paleoposition' u'PDM' u'Pole']


,description,examples,group,label,notes,position,previous_columns,type,unit,urls,validations
age_high,"Location inferred age, High range",NaN,Age,Inferred Age High,NaN,36,"[{u'column': u'average_age_high', u'table': u'...",Number,Custom,NaN,"[min(""age_low""), requiredUnlessTable(""sites"")]"
age_low,"Location inferred age, Low range",NaN,Age,Inferred Age Low,NaN,35,"[{u'column': u'average_age_low', u'table': u'p...",Number,Custom,NaN,"[max(""age_high""), requiredUnlessTable(""sites"")]"
age_unit,"Location inferred age, Age unit",NaN,Age,Inferred Age Unit,NaN,37,"[{u'column': u'average_age_unit', u'table': u'...",String,NaN,NaN,"[cv(""age_unit""), requiredUnlessTable(""sites"")]"
reversal_test,Classification and result of the reversal test,NaN,Direction,Reversal Test,NaN,52,"[{u'column': u'reversal_test', u'table': u'pma...",String,Flag,NaN,"[cv(""reversal_test"")]"
fold_test_significance,Significance level achieved in tilt correction...,NaN,Direction,Fold Test Significance,NaN,48,"[{u'column': u'fold_test_significance', u'tabl...",Number,%,NaN,"[min(0), max(100)]"


In [12]:
# get headers the way we do them in the current builder.py
# not sure we will actually want to do it like this as we update magic_gui.py and pmag_gui.py
cond = location['validations'].map(lambda x: 'required()' in str(x))

reqd_loc_headers = [str(i) for i in location[cond].index]
all_loc_headers = [str(i) for i in location['validations'].index if i not in reqd_loc_headers]
headers = [[], reqd_loc_headers, all_loc_headers] # this is basically how self.headers is organizaed now in builder.py


print headers

[[], ['location_name', 'location_type'], ['age_high', 'age_low', 'age_unit', 'analyst_names', 'citations', 'conglomerate_test', 'contact_test', 'continent_ocean', 'country', 'criteria_names', 'description', 'dic_inc', 'dir_alpha95', 'dir_dec', 'dir_k', 'dir_k_ratio', 'dir_n_samples', 'dir_n_sites', 'dir_polarity', 'dir_r', 'dir_tilt_correction', 'elevation_high', 'elevation_low', 'expedition_description', 'expedition_leg', 'expedition_name', 'expedition_ship', 'expedition_url', 'experiment_names', 'external_database_ids', 'fold_test', 'fold_test_significance', 'geologic_classes', 'geological_province_sections', 'lat_n', 'lat_s', 'lithologies', 'location_name_alternatives', 'lon_e', 'lon_w', 'method_codes', 'ocean_sea', 'padm', 'padm_n_sites', 'padm_sigma', 'paleolat', 'paleolat_sigma', 'paleolon', 'paleolon_sigma', 'pdm', 'pdm_n_sites', 'pdm_sigma', 'pi_names', 'plate_blocks', 'pole_alpha95', 'pole_antipodal_angle', 'pole_bc_q', 'pole_comp_name', 'pole_conf', 'pole_dm', 'pole_dp', 'pol

## Importing datamodel module

In [13]:
import pmagpy.data_model3 as dm
reload(dm)

data_model = dm.DataModel()
data_model.dm['locations']

,description,examples,group,label,notes,position,previous_columns,type,unit,urls,validations
age,Location inferred age,NaN,Age,Inferred Age,NaN,38,"[{u'column': u'average_age', u'table': u'pmag_...",Number,Custom,NaN,"[requiredUnless(""age_low"",""age_high""), require..."
age_high,"Location inferred age, High range",NaN,Age,Inferred Age High,NaN,41,"[{u'column': u'average_age_high', u'table': u'...",Number,Custom,NaN,"[min(""age_low""), requiredUnless(""age"")]"
age_low,"Location inferred age, Low range",NaN,Age,Inferred Age Low,NaN,40,"[{u'column': u'average_age_low', u'table': u'p...",Number,Custom,NaN,"[max(""age_high""), requiredUnless(""age"")]"
age_sigma,"Location inferred age, Uncertainty",NaN,Age,Inferred Age Sigma,Standard error or standard deviation at one sigma,39,"[{u'column': u'average_age_sigma', u'table': u...",Number,Custom,NaN,[min(0)]
age_unit,"Location inferred age, Unit",NaN,Age,Inferred Age Unit,NaN,42,"[{u'column': u'average_age_unit', u'table': u'...",String,NaN,NaN,"[cv(""age_unit""), required()]"
analysts,Colon-delimited list of EarthRef handles and/o...,[@user1:@user2:Not A. Member <no.earthref.hand...,Metadata,Analyst Names,NaN,97,"[{u'column': u'er_analyst_mail_names', u'table...",List,NaN,NaN,"[type(""users"")]"
citations,Colon-delimited list of citation DOIs,"[10.1029/92JB01202, 10.1029/2003GC000635:This ...",Result,Citation Names,NaN,12,"[{u'column': u'er_citation_names', u'table': u...",List,NaN,NaN,"[type(""references""), requiredUnlessTable(""site..."
conglomerate_test,Classification and result of the (intra-format...,NaN,Direction,Conglomerate Test,NaN,56,"[{u'column': u'conglomerate_test', u'table': u...",String,Flag,NaN,"[cv(""conglomerate_test"")]"
contact_test,Classification and result of the (inverse) con...,NaN,Direction,Baked Contact Test,NaN,57,"[{u'column': u'contact_test', u'table': u'pmag...",String,Flag,NaN,"[cv(""contact_test"")]"
continent_ocean,Name for continent or ocean island region,NaN,Geography,Continent or Ocean Island Region,NaN,33,"[{u'column': u'continent_ocean', u'table': u'e...",String,NaN,NaN,"[cv(""continent_ocean"")]"


## Low-level builder stuff with 3.0 -- (but probably irrelevant now that we are switching to MagicDataFrame and Contributions)

In [14]:
os.chdir('/Users/nebula/Python/PmagPy')

from pmagpy import validate_upload
import pmagpy.builder as builder

In [15]:
reload(builder)
reload(validate_upload)
dmodel = full
b = builder.ErMagicBuilder('/Users/nebula/Python/PmagPy/3_0', dmodel)
b.data_model['locations']


loc = builder.Location('location', None, b.data_model)
site = builder.Site('a_site', loc, b.data_model)
samp = builder.Sample('heyo', site, b.data_model)
spec = builder.Specimen('hi', site, b.data_model)


In [16]:
# all of the specimen methods are working

reload(builder)
b.specimens = []
spec = b.add_specimen('new_spec', None, {'geologic_classes': "class1:class2"})
print b.specimens
print spec.data
b.change_specimen('new_spec', 'cool_spec', 'sample', {'geologic_types': "type1:type2"})
print spec.data
b.change_specimen('cool_spec', 'cool_spec', 'sample', {'geologic_types': "type3"}, replace_data=True)
print spec.data
b.delete_methods['specimen']('cool_spec')
print b.specimens
print b.samples

[specimen: new_spec]
{'geologic_classes': 'class1:class2'}
-W- sample is not a currently existing sample.
Creating a new sample named: sample 
{'geologic_classes': 'class1:class2', 'geologic_types': 'type1:type2'}
{'geologic_types': 'type3'}
[]
[sample: sample]


In [17]:
reload(builder)
b = builder.ErMagicBuilder('/Users/nebula/Python/PmagPy/3_0', dmodel)

spec = b.add_specimen('spec', None, {'geologic_classes': 'awesome'}, ['Geology', 'Specimen'])
spec.groups

l = []
data_type = 'specimens'
# add headers that are required for a group
for group in spec.groups:
    l.extend(list(b.data_model[data_type][b.data_model[data_type]['group'] == group].index))

# add cols that are req'd no matter what        
cond = b.data_model[data_type]['validations'].map(lambda x: 'required()' in str(x))
l.extend(list(b.data_model[data_type][cond].index))

# how to deal with the requiredUnless, requiredIf stuff.... : \

b.data_model[data_type]['validations']['age']
#b.data_model['specimens']['group'].unique()



[u'requiredUnless("age_low","age_high")', u'requiredIf("age_sigma")']

In [18]:
b.data_model['specimens'].ix['age']['type']

u'Number'

## Using the data_model module

In [19]:
import pmagpy.data_model3 as data_model


In [20]:
dir(data_model)
reload(data_model)

model_container = data_model.DataModel()

dm = model_container.dm
locs = dm['locations']
#dm['locations']['validations'].str.join(", ")
locs['str_validations'] = locs['validations'].str.join(", ")
locs[locs['str_validations'].str.contains("required\(\)").fillna(False)]

,description,examples,group,label,notes,position,previous_columns,type,unit,urls,validations,str_validations
age_unit,"Location inferred age, Unit",NaN,Age,Inferred Age Unit,NaN,42,"[{u'column': u'average_age_unit', u'table': u'...",String,NaN,NaN,"[cv(""age_unit""), required()]","cv(""age_unit""), required()"
geologic_classes,Colon-delimited list of geologic classes,NaN,Geology,Geologic Classes,NaN,18,"[{u'column': u'location_class', u'table': u'er...",List,NaN,NaN,"[cv(""class""), required()]","cv(""class""), required()"
lat_n,Northernmost latitude of the collection of sites,NaN,Geography,Northernmost Latitude,NaN,27,"[{u'column': u'location_end_lat', u'table': u'...",Number,Degrees,NaN,"[min(-90), max(90), min(""lat_s""), required()]","min(-90), max(90), min(""lat_s""), required()"
lat_s,Southernmost latitude of the collection of sites,NaN,Geography,Southernmost Latitude,NaN,26,"[{u'column': u'location_begin_lat', u'table': ...",Number,Degrees,NaN,"[min(-90), max(90), max(""lat_n""), required()]","min(-90), max(90), max(""lat_n""), required()"
lithologies,Colon-delimited list of lithologies or archeol...,NaN,Geology,Lithologies,NaN,19,"[{u'column': u'location_lithology', u'table': ...",List,NaN,NaN,"[cv(""lithology""), required()]","cv(""lithology""), required()"
location,"Name for location, dredge or drill site","[San Francisco Volcanic Province, Dredge AMAT0...",Names,Location Name,NaN,1,"[{u'column': u'er_location_name', u'table': u'...",String,NaN,NaN,[required()],required()
location_type,Location type,NaN,Location,Location Type,NaN,6,"[{u'column': u'location_type', u'table': u'er_...",String,NaN,NaN,"[cv(""location_type""), required()]","cv(""location_type""), required()"
lon_e,Easternmost longitude of the collection of sites,NaN,Geography,Easternmost Longitude,NaN,29,"[{u'column': u'location_end_lon', u'table': u'...",Number,Degrees,NaN,"[min(0), max(360), required()]","min(0), max(360), required()"
lon_w,Westernmost longitude of the collection of sites,NaN,Geography,Westernmost Longitude,NaN,28,"[{u'column': u'location_begin_lon', u'table': ...",Number,Degrees,NaN,"[min(0), max(360), required()]","min(0), max(360), required()"


In [21]:
print model_container.get_groups('sites')
#print type(model_container.get_headers('sites', 'Age'))

[u'Age', u'Metadata', u'Anisotropy', u'Geology', u'Result', u'Geography', u'Direction', u'Names', u'Site', u'Paleointensity', u'Magnetization', u'Paleoposition', u'VADM', u'VDM', u'VGP']


## Controlled vocabularies

In [22]:
# make sure various pieces of the controlled vocabulary stuff works

import pmagpy.controlled_vocabularies3 as cv
import pmagpy.data_model3 as dm
import numpy as np
reload(dm)
reload(cv)
#print dir(cv)


#print dir(cv.vocab)
vc = cv.Vocabulary()
all_codes, code_types = vc.get_meth_codes()
vc.get_tiered_meth_category('other', all_codes, code_types)
data = vc.get_controlled_vocabularies()

vc.get_tiered_meth_category_offline()
vc.get_all_vocabulary()



def get_cv_from_list(lst):
    """
    If there is a controlled vocabulary
    """
    try:
        for i in lst:
            if "cv(" in i:
                return i[4:-2]
    except TypeError:
        return None
    else:
        return None

data_model = dm.DataModel()
dir(data_model)
data_model.dm['sites']
site_dm = data_model.dm['sites']
site_dm['vocab_name'] = site_dm['validations'].apply(get_cv_from_list)
site_dm[['vocab_name', 'validations']][site_dm['vocab_name'].notnull()]


dir(vc)
print site_dm.ix['geologic_classes']['validations']
vc.vocabularies['geologic_classes'][:5]
vc.vocabularies['age_unit']
vc.vocabularies

-I- Importing controlled vocabularies from https://earthref.org
-I- Importing controlled vocabularies from https://earthref.org
[u'cv("class")', u'required()']


age_unit                 [Ga, Ka, Ma, Years AD (+/-), Years BP, Years C...
alteration                 [Altered, High, Mild, Severe, Trace, Unaltered]
alteration_type          [Acid Leaching, Acid Oxidation, Acid Sulphate,...
aniso_s_unit             [SI, Am^2, bulk in measurements table, Normali...
aniso_type                                         [AMS, AARM, ATRM, AIRM]
assemblage               [Aggregate, In Situ, Mineral Separate, Polycry...
mineral_assemblage       [Aggregate, In Situ, Mineral Separate, Polycry...
int_scat                                  [True, False, true, false, 0, 1]
is_reviewed                               [True, False, true, false, 0, 1]
is_validated                              [True, False, true, false, 0, 1]
geologic_classes         [Archeologic, Extraterrestrial, Extrusive, Ign...
conglomerate_test                 [+, -, G+, G-, Go, IG+, IG-, IGo, ND, o]
contact_test                      [+, -, C+, C-, Co, IC+, IC-, ICo, ND, o]
continent_ocean          

In [23]:
import pmagpy.controlled_vocabularies3 as cv
import pmagpy.data_model3 as dm
import numpy as np
import pandas as pd
reload(dm)
reload(cv)
#print dir(cv)


#print dir(cv.vocab)
vocab = cv.Vocabulary()
vocabulary, possible_vocabulary = vocab.get_controlled_vocabularies()
vocabulary.head()


-I- Importing controlled vocabularies from https://earthref.org


age_unit           [Ga, Ka, Ma, Years AD (+/-), Years BP, Years C...
alteration           [Altered, High, Mild, Severe, Trace, Unaltered]
alteration_type    [Acid Leaching, Acid Oxidation, Acid Sulphate,...
aniso_s_unit       [SI, Am^2, bulk in measurements table, Normali...
aniso_type                                   [AMS, AARM, ATRM, AIRM]
dtype: object

In [24]:
from SPD.mapping import map_magic
reload(map_magic)
dir(map_magic)

x = map_magic.magic2spd_map.pop('specimen_YT')
print map_magic.mapping.__doc__#(map_magic.magic2spd_map
d = dict(zip(map_magic.magic2spd_map.keys(), range(len(map_magic.magic2spd_map.keys()))))
print d
d2 = map_magic.mapping(d, map_magic.magic2spd_map)
print d2


    takes in a dictionary and a mapping which contains new key names,
    and returns a new dictionary with the updated key names, i.e.:
    dictionary = {'a': 1, 'b': 2, 'c': 3}
    mapping = {'a': 'aa', 'c': 'cc'}
    mapped_dictionary = mapping(dictionary, mapping)
    mapped_dictionary = {'aa': 1, b, 2, 'cc': 3}
    
{'specimen_theta': 0, 'fail_ptrm_beta_box_scatter': 1, 'specimen_int_dang': 2, 'specimen_dec': 37, 'specimen_mdev': 5, 'specimen_drat': 52, 'lab_dc_field': 55, 'specimen_k_prime_sse': 49, 'specimen_frac': 8, 'measurement_step_max': 10, 'specimen_PCA_sigma_max': 12, 'specimen_PCA_sigma_min': 53, 'specimen_drats': 40, 'specimen_PCA_sigma_int': 39, 'specimen_b_sigma': 4, 'specimen_ptrms_inc': 17, 'specimen_r_sq': 23, 'specimen_mdrat': 58, 'specimen_dac': 19, 'specimen_dck': 20, 'specimen_gamma': 21, 'specimen_scat_bounding_line_high': 11, 'specimen_int_n': 18, 'specimen_z_md': 24, 'specimen_ac_n': 61, 'specimen_scat_bounding_line_low': 25, 'specimen_inc': 9, 'specimen_in

In [25]:
#print vc.possible_vocabularies
vc.vocabularies['age_unit']

[u'Ga',
 u'Ka',
 u'Ma',
 u'Years AD (+/-)',
 u'Years BP',
 u'Years Cal AD (+/-)',
 u'Years Cal BP']

In [26]:
# working on criteria for lisa

import os
import pmagpy.new_builder as nb
wdir = os.path.join(os.getcwd(), "3_0", "McMurdo")
contribution = nb.Contribution(wdir, read_tables=['criteria'])
crit_container = contribution.tables['criteria']
crit_data = crit_container.df
crit_data = crit_data[crit_data['criterion'].str.contains('IE-')==True] # fish out all the relavent data
crit_dict = dict(crit_data['criterion_value'])
{key.split(".")[1]: value for key, value in crit_dict.items()}

-I- Importing controlled vocabularies from https://earthref.org


{'int_b_beta': '0.14',
 'int_dang': '13.0',
 'int_drats': '17.0',
 'int_f': '0.7',
 'int_fvds': '0.3',
 'int_md': '15.0',
 'int_n_ptrm': '2.0',
 'int_n_specimens': '1.0',
 'int_q': '1.0'}